In [ ]:
from transformers import GPT2Tokenizer, T5ForConditionalGeneration
from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm

import wandb

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

from torch.amp import autocast

from accelerate import Accelerator

In [ ]:
data_tr = load_dataset("IlyaGusev/ru_turbo_alpaca", split="train")

In [ ]:
checkpoint = 'ai-forever/FRED-T5-1.7B'
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint, eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

In [ ]:
emb = []
for i in tqdm(range(len(data_tr))):
    instruction = data_tr[i]["instruction"]
    input = data_tr[i]["input"]
    output = data_tr[i]["output"]
    
    train_row = f"[BIN]{instruction}[EIN] [BEI]{input}[EEI] [BOO]{output}[EOO] {tokenizer.eos_token}"
    emb.append(train_row)

In [ ]:
import jsonlines

with jsonlines.open('alpaca_ds.jsonl', mode='w') as writer:
    writer.write(emb)